# Instala las librerías necesarias:

`pip install selenium beautifulsoup4`

In [ ]:
!pip install selenium beautifulsoup4

# Instalar Google Chrome en Linux

In [ ]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb

In [ ]:
!sudo dpkg -i google-chrome-stable_current_amd64.deb

# Bajar ChromeDriver

[Acá](https://chromedriver.chromium.org/downloads) puedes encontrar las diferentes versiones de ChromeDriver. Chequea tu versión de google Chrome para bajar el driver correcto

In [ ]:
!google-chrome --version

In [ ]:
!ls

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [ ]:
driver = webdriver.Chrome(executable_path="./chromedriver")
driver.get("https://domicilios.com/restaurantes/bogota/ppc-suba-menu?t=comentarios")

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
info = []
for review in soup.select("#reviewList > li"):
    text = review.select("[itemprop='description']")[0].get_text()
    author = review.select("div.small")[0].get_text()
    previousOrders = review.select("div.small")[1]["title"].split()[0]
    previousReviews = review.select("div.small")[2]["title"].split()[0]
    date = review.select("[itemprop='datePublished']")[0].get_text()
    worstRating = review.select("[itemprop='worstRating']")[0]["content"]
    ratingValue = review.select("[itemprop='ratingValue']")[0]["content"]
    bestRating = review.select("[itemprop='bestRating']")[0]["content"]
    infoReview = dict(text=text, author=author, previousOrders=previousOrders, previousReviews=previousReviews,
                     date=date, worstRating=worstRating, ratingValue=ratingValue, bestRating=bestRating)
    info.append(infoReview)

In [ ]:
print(info)

In [ ]:
button = driver.find_element_by_css_selector("#reviewListContainer > footer > ul > li.arrow.next > button")
print(button.get_attribute("data-page"))
button.click()

In [ ]:
driver.quit()

# Todo funciona perfecto! :-D

Ahora volvamos esto una función para iterar sobre todas las páginas de los comentarios de nuestro restaurante de ejemplo

In [ ]:
import time

In [ ]:
def download_domicilios(website):
    driver = webdriver.Chrome(executable_path="./chromedriver")
    driver.get(website)
    driver.implicitly_wait(100)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    info = []
    pages = []
    while True:
        try:
            button = driver.find_element_by_css_selector("#reviewListContainer > footer > ul > li.arrow.next > button")
            numPage = button.get_attribute("data-page")
            print('Currently at page {0}'.format(numPage))
            if numPage in pages:
                driver.quit()
                raise ValueError("numPage {0} was already visited".format(numPage))
            else:
                pages.append(numPage)
            for review in soup.select("#reviewList > li"):
                text = review.select("[itemprop='description']")[0].get_text()
                author = review.select("div.small")[0].get_text()
                previousOrders = review.select("div.small")[1]["title"].split()[0]
                previousReviews = review.select("div.small")[2]["title"].split()[0]
                date = review.select("[itemprop='datePublished']")[0].get_text()
                worstRating = review.select("[itemprop='worstRating']")[0]["content"]
                ratingValue = review.select("[itemprop='ratingValue']")[0]["content"]
                bestRating = review.select("[itemprop='bestRating']")[0]["content"]
                infoReview = dict(text=text, author=author, previousOrders=previousOrders, previousReviews=previousReviews,
                                 date=date, worstRating=worstRating, ratingValue=ratingValue, bestRating=bestRating)
                info.append(infoReview)
            time.sleep(3)
            button.click()
        except Exception as e:
            print(e)
            break
    return info

In [ ]:
info = download_domicilios("https://domicilios.com/restaurantes/bogota/ppc-suba-menu?t=comentarios")

In [ ]:
len(info)

In [ ]:
import json

with open("ppc_suba_comentarios.json", "w") as f:
    json.dump(info, f)